In [2]:
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")

cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT,
    price REAL
);
""")

cursor.execute("""
CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    quantity INTEGER,
    sale_date TEXT,
    FOREIGN KEY(product_id) REFERENCES products(id)
);
""")

# Insert sample data into products
cursor.executemany("""
INSERT INTO products (id, name, category, price) VALUES (?, ?, ?, ?);
""", [
    (1, "Laptop", "Electronics", 1200),
    (2, "Headphones", "Electronics", 150),
    (3, "Coffee Maker", "Home", 80),
    (4, "Desk Chair", "Furniture", 200)
])

# Insert sample data into sales
cursor.executemany("""
INSERT INTO sales (sale_id, product_id, quantity, sale_date) VALUES (?, ?, ?, ?);
""", [
    (1, 1, 2, "2024-01-03"),
    (2, 2, 5, "2024-01-04"),
    (3, 3, 1, "2024-01-04"),
    (4, 1, 1, "2024-01-05"),
    (5, 4, 3, "2024-01-06")
])

conn.commit()

### Query all rows from the products and sales tables

In [3]:
pd.read_sql_query("SELECT * FROM products;", conn)

,id,name,category,price
0,1,Laptop,Electronics,1200.0
1,2,Headphones,Electronics,150.0
2,3,Coffee Maker,Home,80.0
3,4,Desk Chair,Furniture,200.0


In [4]:
pd.read_sql_query("SELECT * FROM sales;", conn)

,sale_id,product_id,quantity,sale_date
0,1,1,2,2024-01-03
1,2,2,5,2024-01-04
2,3,3,1,2024-01-04
3,4,1,1,2024-01-05
4,5,4,3,2024-01-06


### SQL filter for items from sales where quanity > 2

In [5]:
pd.read_sql_query("SELECT * FROM sales WHERE quantity > 2;", conn)

,sale_id,product_id,quantity,sale_date
0,2,2,5,2024-01-04
1,5,4,3,2024-01-06


### SQL Order By query

In [6]:
pd.read_sql_query("SELECT * FROM sales ORDER BY quantity ASC;", conn)

,sale_id,product_id,quantity,sale_date
0,3,3,1,2024-01-04
1,4,1,1,2024-01-05
2,1,1,2,2024-01-03
3,5,4,3,2024-01-06
4,2,2,5,2024-01-04


In [7]:
pd.read_sql_query("SELECT * FROM sales ORDER BY quantity DESC;", conn)

,sale_id,product_id,quantity,sale_date
0,2,2,5,2024-01-04
1,5,4,3,2024-01-06
2,1,1,2,2024-01-03
3,3,3,1,2024-01-04
4,4,1,1,2024-01-05


### SQL Group By query

In [8]:
pd.read_sql_query("SELECT product_id, SUM(quantity) FROM sales GROUP BY product_id;", conn)

,product_id,SUM(quantity)
0,1,3
1,2,5
2,3,1
3,4,3


In [9]:
pd.read_sql_query("SELECT category, AVG(price) FROM products GROUP BY category;", conn)

,category,AVG(price)
0,Electronics,675.0
1,Furniture,200.0
2,Home,80.0


### SQL JOIN for two datasets

In [10]:
pd.read_sql_query("SELECT * FROM sales JOIN products ON sales.product_id = products.id;", conn)

,sale_id,product_id,quantity,sale_date,id,name,category,price
0,1,1,2,2024-01-03,1,Laptop,Electronics,1200.0
1,2,2,5,2024-01-04,2,Headphones,Electronics,150.0
2,3,3,1,2024-01-04,3,Coffee Maker,Home,80.0
3,4,1,1,2024-01-05,1,Laptop,Electronics,1200.0
4,5,4,3,2024-01-06,4,Desk Chair,Furniture,200.0


### Compute revenue per product using a JOIN + GROUP BY

In [11]:
pd.read_sql_query("SELECT name, SUM(quantity) AS total_units, SUM(quantity*price) AS total_revenue FROM sales JOIN products ON product_id = id GROUP BY name ORDER BY SUM(quantity*price) DESC;", conn)

,name,total_units,total_revenue
0,Laptop,3,3600.0
1,Headphones,5,750.0
2,Desk Chair,3,600.0
3,Coffee Maker,1,80.0


## SQL Mini-Project: Sales Analysis

### Revenue by product category

In [17]:
pd.read_sql_query("SELECT category, SUM(price*quantity) AS revenue_generated from sales JOIN products ON sales.product_id = products.id GROUP BY category;", conn)

,category,revenue_generated
0,Electronics,4350.0
1,Furniture,600.0
2,Home,80.0


### Top selling products by units

In [48]:
pd.read_sql_query("SELECT name, SUM(quantity) AS total_units_sold from sales JOIN products ON sales.product_id = products.id GROUP BY name ORDER BY total_units_sold DESC;", conn)

,name,total_units_sold
0,Headphones,5
1,Laptop,3
2,Desk Chair,3
3,Coffee Maker,1


### Revenue by day

In [51]:
pd.read_sql_query("SELECT sale_date, SUM(quantity*price) AS days_revenue from sales JOIN products ON sales.product_id = products.id GROUP BY sale_date;", conn)

,sale_date,days_revenue
0,2024-01-03,2400.0
1,2024-01-04,830.0
2,2024-01-05,1200.0
3,2024-01-06,600.0


In [52]:
pd.read_sql_query("SELECT sale_date, SUM(quantity*price) AS days_revenue from sales JOIN products ON sales.product_id = products.id GROUP BY sale_date ORDER BY days_revenue DESC LIMIT 1;", conn)

,sale_date,days_revenue
0,2024-01-03,2400.0


### Product performance summary

In [53]:
pd.read_sql_query("SELECT name, SUM(quantity) AS total_units_sold, SUM(quantity*price) AS total_revenue, COUNT(sale_date) AS total_sales from sales JOIN products ON sales.product_id = products.id GROUP BY products.name, sales.product_id, products.price;", conn)

,name,total_units_sold,total_revenue,total_sales
0,Coffee Maker,1,80.0,1
1,Desk Chair,3,600.0,1
2,Headphones,5,750.0,1
3,Laptop,3,3600.0,2
